In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, \
                            EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import h5py

from helpers import TrainValTensorBoard, get_data
from models import get_unet
from losses import dice_loss

Using TensorFlow backend.


In [3]:
LOGDIR='logs'

In [4]:
xtrain, xval, ytrain, yval, \
dtrain, dval, idtrain, idval = get_data()

H, W = 224, 224
lr = 1e-3
loss = dice_loss

BATCH_SIZE = 32
EPOCHS = 1

model = get_unet(H, W, lr, loss)

/Users/brennangebotys/miniconda3/envs/playground/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [5]:
datagen = ImageDataGenerator(zoom_range=0.1,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             vertical_flip=True)
datagen.fit(xtrain)

In [6]:
# define callbacks
lr_plat = ReduceLROnPlateau(monitor='val_dice_coef',
                               factor=0.2,
                               patience=5,
                               verbose=1,
                               min_delta=1e-4,
                               mode='max')
early_stop = EarlyStopping(monitor='val_dice_coef',
                           patience=10,
                           verbose=1,
                           min_delta=1e-4,
                           mode='max')
m_checkpoint = ModelCheckpoint(monitor='val_dice_coef',
                             filepath='model_weights.hdf5',
                             save_best_only=True,
                             mode='max')
tb = TrainValTensorBoard(write_graph=False)
callbacks = [lr_plat, early_stop, m_checkpoint, tb]

In [7]:
model.fit_generator(generator=datagen.flow(xtrain, ytrain, batch_size=BATCH_SIZE),
                    steps_per_epoch=np.ceil(float(len(xtrain)) / float(BATCH_SIZE)),
                    epochs=EPOCHS,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=(xval, yval), 
                    validation_steps=np.ceil(float(len(xval)) / float(BATCH_SIZE)))

Epoch 1/1


KeyboardInterrupt: 

In [2]:
# !ipython nbconvert --to=python train.ipynb